This a document that is create a showing test queries for learning sql and the python libaraies.

GITHUB URL:  <https://github.com/cmsc-vcu/cmsc508-fa2023-hw7-bradley-daniel>


In [ ]:
#| eval: true
#| echo: false
import os
import sys
import pandas as pd
from tabulate import tabulate
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
from sqlalchemy.exc import NoSuchModuleError, OperationalError, ProgrammingError, ArgumentError
from IPython.display import display
from IPython.core.display import Markdown

In [ ]:
#| eval: true
#| echo: false

# modify config_map to reflect credentials needed by this program
config_map = {
    'user':'CMSC508_USER',
    'password':'CMSC508_PASSWORD',
    'host':'CMSC508_HOST',
    'database':'HW7_DB_NAME'
}
# load and store credentials
load_dotenv()
config = {}
for key in config_map.keys():
    config[key] = os.getenv(config_map[key])
flag = False
for param in config.keys():
    if config[param] is None:
        flag = True
        print(f"Missing {config_map[param]} in .env file")
#if flag:
#    sys.exit(1)

In [ ]:
#| eval: true
#| echo: false

# build a sqlalchemy engine string
engine_uri = f"mysql+pymysql://{config['user']}:{config['password']}@{config['host']}/{config['database']}"

# create a database connection.  THIS IS THE ACTUAL CONNECTION!
try:
    cnx = create_engine(engine_uri)
except ArgumentError as e:
    if e == NoSuchModuleError:
        print(f"create_engine: No Such Module Error: {e}")
    print(f"create_engine: Argument Error: {e}")
    sys.exit(1)
except Exception as e:
    print(f"create_engine: An error occurred: {e}")
    sys.exit(1)

In [ ]:
#| echo: false
#| eval: true

# Do a quick test of the connection and trap the errors better!
try:
    databases = pd.read_sql("show databases",cnx)
except ImportError as e:
    print(f"Error: {e}")
except OperationalError as e:
    print(f"Database/SQL Error:\n{str(e)}\n")
except ProgrammingError as e:
    print(f"Programming Error:\n{str(e)}\n")
except Exception as e:
    print(f"An error occurred:\n{str(e)}\n")

## Overview and description

This is a database that store data about skils which people have those skills. It also stores roles and the people that have those roles.

## Crows-foot diagram

This is a Crows-foot diagram of the database showing the relations between people and roles and skills.


```{mermaid}
%%| echo: false
erDiagram
    People  {
        int id 
        string first_name 
        string last_name
        string email 
        string linkedin_url
        string headshot_url 
        string dicord_handle
        string brief_bio
        date date_joined
    }
    Skills {
        int id
        string name
        string description
        string tag
        date required_time
    }
    Roles {
        int id 
        string name 
        int sort_priority
    }
    People }o--o{ Skills : has
    People }o--o{ Roles : has
```


## Examples of data in the database

For the people, skills, and roles tables, provide a description of each table and it's contents. Then provide a listing of the entire table.

### People table

The *people* table contains elements that describe ... 

Below is a list of data in the *people* table.


In [ ]:
#| echo: false
#| eval: true
sql = f"""
select * from people;
"""
## Add code to list roles table here
try:
    df = pd.read_sql(sql,cnx)
    df
except Exception as e:
    message = str(e)
    print(f"An error occurred:\n\n{message}\n\nIgnoring and moving on.")
    df = pd.DataFrame()
df

### Skills table

The *skills* table contains elements that describe ... 

Below is a list of data in the *skills* table.


In [ ]:
#| echo: false
#| eval: true

## Add code to list roles table here

### Roles table

The *roles* table contains elements that describe ... 

Below is a list of data in the *roles* table.


In [ ]:
#| echo: false
#| eval: true

## Add code to list roles table here

## Sample queries

Let's explore the database!

# List skill names, tags, and descriptions ordered by name

(here a sample query)


In [ ]:
sql = f"""
select * from people
"""

(and here is a sample result from the query)


In [ ]:
#| echo: false
#| eval: true
try:
    df = pd.read_sql(sql,cnx)
    df
except Exception as e:
    message = str(e)
    print(f"An error occurred:\n\n{message}\n\nIgnoring and moving on.")
    df = pd.DataFrame()
df

In [ ]:
#| echo: false
#| eval: true
def query_database(sql):
    try:
        df = pd.read_sql(sql,cnx)
    except Exception as e:
        message = str(e)
        print(f"An error occurred:\n\n{message}\n\nIgnoring and moving on.")
        df = pd.DataFrame()
    return df


### List people names and email addresses ordered by last_name


In [ ]:
sql = f"""
select first_name, last_name from people ORDER BY last_name
"""

In [ ]:
#| echo: false
#| eval: true
query_database(sql)

### List skill names of Person 1


In [ ]:
sql = f"""
select 
    skills.name 'person_1_skills_names' 
from 
    people
    inner join peopleskills on (people.id=peopleskills.people_id)
    inner join skills on (peopleskills.skills_id=skills.id)
where 
    people.last_name='Person 1';
"""

In [ ]:
#| echo: false
#| eval: true
query_database(sql)

### List people names with Skill 6

In [ ]:
sql = f"""
select 
    first_name, last_name
from 
    people
    inner join peopleskills on (people.id=peopleskills.people_id)
    inner join skills on (peopleskills.skills_id=skills.id)
where 
    skills.tag='Skill_6';
"""

In [ ]:
#| echo: false
#| eval: true
query_database(sql)

### List people with a DEVELOPER role


In [ ]:
sql = f"""
select 
    first_name, last_name
from 
    people
    inner join peopleroles on (people.id=peopleroles.people_id)
    inner join roles on (peopleroles.role_id=roles.id)
where 
    roles.name='Developer'
"""

In [ ]:
#| echo: false
#| eval: true
query_database(sql)

### List names and email addresses of people without skills


In [ ]:
sql = f"""
select 
    first_name, last_name, email
from 
    people
    left join peopleskills on (people.id=peopleskills.people_id)
where 
    peopleskills.skills_id is NULL;
"""

In [ ]:
#| echo: false
#| eval: true
query_database(sql)

### List names and tags of unused skills


In [ ]:
sql = f"""
select 
    skills.name, skills.tag
from 
    skills
    left join peopleskills on (skills.id=peopleskills.skills_id)
where 
    peopleskills.people_id is NULL;
"""

In [ ]:
#| echo: false
#| eval: true
query_database(sql)

### List people names and skill names with the BOSS role


In [ ]:
sql = f"""
# select first_name, last_name from people ORDER BY last_name
select 
    people.first_name, people.last_name, skills.name skill_name
from 
    people
    inner join peopleroles on (people.id=peopleroles.people_id)
    inner join roles on (peopleroles.role_id=roles.id)
    inner join peopleskills on (people.id=peopleskills.people_id)
    inner join skills on (peopleskills.skills_id=skills.id)
where 
    roles.name="Boss";
"""

In [ ]:
#| echo: false
#| eval: true
query_database(sql)

### List ids and names of unused roles


In [ ]:
sql = f"""
select 
    roles.id, roles.name
from 
    roles
    left join peopleroles on (roles.id=peopleroles.role_id)
where 
    peopleroles.people_id is NUll;
"""

In [ ]:
#| echo: false
#| eval: true
query_database(sql)

## Reflection

I found that writing the SQL statement is much easier than writing relation algebra. It was so nice to be able to just write a query and test it. I can see how this would SQL is a powerful tool after writing my own queries. I did an internship last summer and if I knew SQL it would have made it a lot easier to implement some of the functionality of my summer project.
